In [2]:
'hcp.recobundles' + f'/sub-{2}/sub-{3}_csd.nii.gz'

'hcp.recobundles/sub-2/sub-3_csd.nii.gz'

In [7]:
from AFQ.api import make_bundle_dict
bundles_waypoint_roi = set(make_bundle_dict().keys())
print(bundles_waypoint_roi)

Dataset is already in place. If you want to fetch it again please first remove the folder /Users/john/AFQ_data/templates 
Dataset is already in place. If you want to fetch it again please first remove the folder /Users/john/AFQ_data/callosum_templates 
{'IFO_R', 'CGC_L', 'FP', 'ATR_L', 'UNC_L', 'FA', 'CST_R', 'HCC_R', 'SLF_L', 'CST_L', 'ILF_R', 'ILF_L', 'IFO_L', 'ATR_R', 'SLF_R', 'HCC_L', 'ARC_R', 'UNC_R', 'CGC_R', 'ARC_L'}


In [6]:
from AFQ.data import read_hcp_atlas_16_bundles
bundles_reco = set(read_hcp_atlas_16_bundles().keys())
print(bundles_reco)

Dataset is already in place. If you want to fetch it again please first remove the folder /Users/john/AFQ_data/hcp_atlas_16_bundles 
{'AF_L', 'F_R', 'CST_L', 'IFOF_R', 'F_L', 'UF_R', 'C_L', 'CST_R', 'UF_L', 'CCMid', 'MCP', 'whole_brain', 'CC_ForcepsMajor', 'CC_ForcepsMinor', 'AF_R', 'C_R', 'IFOF_L'}


In [8]:
print(bundles_waypoint_roi & bundles_reco)

{'CST_L', 'CST_R'}


In [ ]:
# IFOF is IFO
# UNC is UF
# AF is ARC
# CST is CST
# FA and FP maybe related to F_L and F_R

# F-1 score comparing set membership
# dice coefficient between covered voxels (1 or 0 map)
# compare bundle profiles

In [ ]:
# from pyAFQ examples. Performs tractography on HARDI dataset.
"""
==========================
Plotting tract profiles
==========================

An example of tracking and segmenting two tracts, and plotting their tract
profiles for FA (calculated with DTI).

"""
import os.path as op
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
import dipy.data as dpd
from dipy.data import fetcher
import dipy.tracking.utils as dtu
import dipy.tracking.streamline as dts
from dipy.io.streamline import save_tractogram, load_tractogram
from dipy.stats.analysis import afq_profile, gaussian_weights
from dipy.io.stateful_tractogram import StatefulTractogram
from dipy.io.stateful_tractogram import Space

import AFQ.utils.streamlines as aus
import AFQ.data as afd
import AFQ.tractography as aft
import AFQ.registration as reg
import AFQ.dti as dti
import AFQ.segmentation as seg
from AFQ.utils.volume import patch_up_roi
from AFQ.api import make_bundle_dict

def 
print(f"Fetching HCP subject {subject}")
afd.fetch_hcp([subject], 
              profile_name=False,
              aws_access_key_id=hcp_ak,
              aws_secret_access_key=hcp_sk)  
dwi_dir = op.join(afd.afq_home, 'HCP', 'derivatives',
                  'dmriprep', f'sub-{subject}', 'sess-01/dwi')

hardi_fdata = op.join(dwi_dir, f"sub-{subject}_dwi.nii.gz")
hardi_fbval = op.join(dwi_dir, f"sub-{subject}_dwi.bval")
hardi_fbvec = op.join(dwi_dir, f"sub-{subject}_dwi.bvec")

img = nib.load(hardi_fdata)

print("Calculating DTI...")
if not op.exists(f'./{subject}/dti_FA.nii.gz'):
    dti_params = dti.fit_dti(hardi_fdata, hardi_fbval, hardi_fbvec,
                             out_dir=f'./{subject}')
else:
    dti_params = {'FA': f'./{subject}/dti_FA.nii.gz',
                  'params': f'./{subject}/dti_params.nii.gz'}

FA_img = nib.load(dti_params['FA'])
FA_data = FA_img.get_fdata()

templates = afd.read_templates()
bundles = make_bundle_dict()

print("Registering to template...")
MNI_T2_img = dpd.read_mni_template()
if not op.exists(f'./{subject}/mapping.nii.gz'):
    import dipy.core.gradients as dpg
    gtab = dpg.gradient_table(hardi_fbval, hardi_fbvec)
    warped_hardi, mapping = reg.syn_register_dwi(hardi_fdata, gtab)
    reg.write_mapping(mapping, f'./{subject}/mapping.nii.gz')
else:
    mapping = reg.read_mapping(f'./{subject}/mapping.nii.gz', img, MNI_T2_img)


print("Tracking...")
if not op.exists(f'./{subject}/dti_streamlines.trk'):
    seed_roi = np.zeros(img.shape[:-1])
    for name in bundle_names:
        for hemi in ['_R', '_L']:
            for roi in bundles[name + hemi]['ROIs']:
                warped_roi = patch_up_roi(
                    (mapping.transform_inverse(
                        roi.get_data().astype(np.float32),
                     interpolation='linear')) > 0)

                # Add voxels that aren't there yet:
                seed_roi = np.logical_or(seed_roi, warped_roi)

    nib.save(nib.Nifti1Image(seed_roi.astype(float), img.affine), f'./{subject}/seed_roi.nii.gz')
    streamlines = aft.track(dti_params['params'], #seed_mask=seed_roi,
                            stop_mask=FA_data, stop_threshold=0.1)

    sft = StatefulTractogram(streamlines, img, Space.RASMM)
    save_tractogram(sft, f'./{subject}/dti_streamlines.trk',
                    bbox_valid_check=False)
else:
    tg = load_tractogram(f'./{subject}/dti_streamlines.trk', img)
    streamlines = tg.streamlines

streamlines = dts.Streamlines(
    dtu.transform_tracking_output(streamlines,
                                  np.linalg.inv(img.affine)))

print("Segmenting...")
segmentation_reco = seg.Segmentation(algo='reco',
                                     model_clust_thr=20,
                                     reduction_thr=20,
                                     b0_threshold=50,
                                     return_idx=True)
bundle_names_reco = ['CST',
                     'UF',
                     'AF',
                     'IFOF']
bundle_names_reco = api.make_bundle_dict(bundle_names=bundle_names_reco, seg_algo='reco')
fiber_groups_reco = segmentation_reco.segment(bundle_names_reco, streamlines, hardi_fdata, hardi_fbval, hardi_fbvec,
                                              mapping=mapping, reg_template=MNI_T2_img)

segmentation_afq = seg.Segmentation(algo='afq', return_idx=True)
bundle_names_afq = ['CST',
                    'UNC',
                    'ARC',
                    'IFO']
bundle_names_afq = api.make_bundle_dict(bundle_names=bundle_names_afq, seg_algo='afq')
fiber_groups_afq = segmentation_afq.segment(bundle_names_afq, streamlines, hardi_fdata, hardi_fbval, hardi_fbvec,
                                            mapping=mapping, reg_template=MNI_T2_img)

print("Cleaning...")
for kk in fiber_groups:
    fiber_groups_reco[kk] = seg.clean_fiber_group(fiber_groups_reco[kk])
    fiber_groups_afq[kk] = seg.clean_fiber_group(fiber_groups_afq[kk])
    
print("Extracting tract profiles...")
profiles = []
for kk in fiber_groups_reco:
    weights = gaussian_weights(fiber_groups_reco[kk])
    profile = afq_profile(FA_data, fiber_groups_reco[kk],
                          np.eye(4), weights=weights)
    for ii in profile:
        # Subject, Bundle, node, method, metric (FA, MD), value
        profiles.append(subject, kk, ii, 'reco', 'FA', profile[ii])
for kk in fiber_groups_afq:
    weights = gaussian_weights(fiber_groups_afq[kk])
    profile = afq_profile(FA_data, fiber_groups_afq[kk],
                          np.eye(4), weights=weights)
    for ii in profile:
        # Subject, Bundle, node, method, metric (FA, MD), value
        profiles.append(subject, kk, ii, 'afq', 'FA', profile[ii])

profiles = pd.DataFrame(data=profiles, columns=["Subject", "Bundle", "Node", "Method", "Metric", "Value"])
profiles.to_csv(f"./{subject}/profiles.csv")